# Getting Started

First, we import the sys module, which helps us work with system-specific parameters and functions in Python. By appending the parent directory (../) to the module search path, we enable Python to search for modules in that directory whenever we try to import them. This allows us to conveniently access and use modules located in the parent directory within our code.

In [1]:
import sys
# Add parent directory to the module search path
sys.path.append('../')

In [2]:
import time
import torch
from tqdm.notebook import trange

Now, we import the custom functions and models we have defined in our scripts folders. I have did this to easily access and use functions necessary for training and evaluating the models, so that the notebook is easier to follow without all function cluster.

In [3]:
from scripts.utils import set_seed, count_parameters, \
                            epoch_time, df_results
from scripts.data import get_transforms, load_cifar10_datasets, \
                            split_train_validation, create_data_loaders
from scripts.models import LeNet, LeNet_leaky, LeNet_sigmoid
from scripts.train import train_model, evaluate

We set a seed number to make the results reproducible

In [4]:
SEED = 1773
set_seed(SEED)

# Loading data

We can now load the dataset and split it into train, validation, and test sets. Actually, in CIFAR10 dataset, train and test datasets are already seperated so we just need to load them and split the train set into training and validaiton sets.

In [5]:
train_transform, test_transform = get_transforms()
train_data, test_data = load_cifar10_datasets(train_transform=train_transform, 
                                              test_transform=test_transform)

print(f'Number of training examples and shape: {train_data.data.shape}')
print(f'Number of testing examples and shape: {test_data.data.shape}')

Files already downloaded and verified
Files already downloaded and verified
Number of training examples and shape: (50000, 32, 32, 3)
Number of testing examples and shape: (10000, 32, 32, 3)


In [6]:
train_data, valid_data = split_train_validation(train_data, 0.9, test_transform)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')

Number of training examples: 45000
Number of validation examples: 5000


# Model Training

Although we explored different parameters in each model, they all have a similar architecture which is based on LeNet. However, since the original LeNet was developed to work on grey-scale we need to arrange the input dimensions. Beacuse we are working with CIFAR10 dataset and it contains RGB images.

## Model-0

The first model is a regular implementation of LeNet. I have only changed the input dimension to 3 since, the original LeNet has been developed to work on grey-scale images. But CIFAR10 dataset contains RGB images.

This base model will be compared to other models with different hyperparameters. The specs of this model are as follows:

 - Batch Size : 64
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD) w/ Momentum = 0.9, Learning rate = 0.001
 - Activation function: ReLu
 - Epoch : 32

In [7]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
# Define the batch size for the data loaders
BATCH_SIZE = 64

# Use the create_data_loaders function to create data loaders for the training, validation, and testing datasets
# Each loader will return data in batches of size defined by BATCH_SIZE
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)

# Define the loss function (criterion) for the model
# In this case, we are using Cross Entropy Loss which is typically used for classification tasks
criterion = torch.nn.CrossEntropyLoss()

# Define the optimization algorithm for training the model
# We are using Stochastic Gradient Descent (SGD) with a learning rate of 0.001 and momentum of 0.9
# The optimizer adjusts the model's parameters based on the calculated gradients during backpropagation
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)


In [10]:
# Check if CUDA is available and set the device to CUDA if available; otherwise, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Print the device being used
print(f"Current device is : {device}")


Current device is : cuda


In [11]:
# Set the number of epochs for training
EPOCHS = 32

# Initialize lists to store the losses and accuracies for each epoch
train_losses = []
valid_losses = []
valid_accs = []
train_accs = []

# Set the best validation loss to infinity for tracking the best model
best_valid_loss = float('inf')

# Move the model and the loss function (criterion) to the GPU (if available)
model = model.to(device)
criterion = criterion.to(device)

# For each epoch, train the model and evaluate it
for epoch in trange(EPOCHS):
  
  # Record the start time of the epoch
  start_time = time.monotonic()

  # Train the model and get the loss and accuracy
  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  
  # Evaluate the model and get the loss and accuracy
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  # Store the losses and accuracies
  train_losses.append(train_loss)
  valid_losses.append(valid_loss)
  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  # If the current model has the best validation loss, save it
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-00.pt')

  # Record the end time of the epoch and calculate the total time
  end_time = time.monotonic()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  # Print the epoch details
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# Save the training statistics to a DataFrame and store it to a CSV file
train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-00-train-stats")



  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 30s
	Train Loss: 2.300 | Train Acc: 10.69%
	 Val. Loss: 2.293 |  Val. Acc: 15.05%


Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 2.212 | Train Acc: 16.43%
	 Val. Loss: 2.067 |  Val. Acc: 23.42%


Epoch: 03 | Epoch Time: 0m 20s
	Train Loss: 1.987 | Train Acc: 25.95%
	 Val. Loss: 1.902 |  Val. Acc: 31.76%


Epoch: 04 | Epoch Time: 0m 20s
	Train Loss: 1.812 | Train Acc: 33.56%
	 Val. Loss: 1.719 |  Val. Acc: 36.61%


...

Epoch: 28 | Epoch Time: 0m 20s
	Train Loss: 0.951 | Train Acc: 66.40%
	 Val. Loss: 1.054 |  Val. Acc: 63.09%


Epoch: 29 | Epoch Time: 0m 19s
	Train Loss: 0.937 | Train Acc: 66.93%
	 Val. Loss: 1.061 |  Val. Acc: 62.22%


Epoch: 30 | Epoch Time: 0m 19s
	Train Loss: 0.921 | Train Acc: 67.72%
	 Val. Loss: 1.073 |  Val. Acc: 62.16%


Epoch: 31 | Epoch Time: 0m 20s
	Train Loss: 0.906 | Train Acc: 68.14%
	 Val. Loss: 1.062 |  Val. Acc: 62.42%


Epoch: 32 | Epoch Time: 0m 19s
	Train Loss: 0.896 | Train Acc: 68.39%
	 Val. Loss: 1.085 |  Val. Acc: 62.06%


In [12]:
# Load the model weights from the best model obtained during training
model.load_state_dict(torch.load('../results/LeNet-00.pt'))

# Evaluate the model on the test set and obtain the loss and accuracy
test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

# Print the test loss and accuracy
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 1.056 | Test Acc: 62.93%


## Comparing Activation Functions

The following models have the same hyperparameters. Only the activation function used in the LeNet model has been changed to be able to compare them.

### Model_01

 - Batch Size : 64
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: Leaky ReLu
 - Epoch : 32

In [13]:
model = LeNet_leaky()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet_leaky(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [14]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [16]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-01.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-01-train-stats")

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 2.301 | Train Acc: 10.12%
	 Val. Loss: 2.295 |  Val. Acc: 9.91%


Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 2.236 | Train Acc: 17.83%
	 Val. Loss: 2.086 |  Val. Acc: 25.04%


Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 1.976 | Train Acc: 28.65%
	 Val. Loss: 1.884 |  Val. Acc: 31.27%


Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 1.785 | Train Acc: 34.96%
	 Val. Loss: 1.711 |  Val. Acc: 36.89%


...

Epoch: 28 | Epoch Time: 0m 18s
	Train Loss: 0.944 | Train Acc: 67.03%
	 Val. Loss: 1.101 |  Val. Acc: 61.16%


Epoch: 29 | Epoch Time: 0m 18s
	Train Loss: 0.931 | Train Acc: 67.51%
	 Val. Loss: 1.100 |  Val. Acc: 61.45%


Epoch: 30 | Epoch Time: 0m 18s
	Train Loss: 0.913 | Train Acc: 68.22%
	 Val. Loss: 1.072 |  Val. Acc: 61.53%


Epoch: 31 | Epoch Time: 0m 20s
	Train Loss: 0.901 | Train Acc: 68.58%
	 Val. Loss: 1.063 |  Val. Acc: 63.05%


Epoch: 32 | Epoch Time: 0m 18s
	Train Loss: 0.882 | Train Acc: 69.27%
	 Val. Loss: 1.097 |  Val. Acc: 61.61%


In [17]:
model.load_state_dict(torch.load('../results/LeNet-01.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.049 | Test Acc: 63.52%


### Model_02

 - Batch Size : 64
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: Sigmoid
 - Epoch : 32

In [18]:
model = LeNet_sigmoid()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet_sigmoid(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [19]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [21]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-02.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-02-train-stats")

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 17s
	Train Loss: 2.304 | Train Acc: 9.95%
	 Val. Loss: 2.304 |  Val. Acc: 9.51%


Epoch: 02 | Epoch Time: 0m 19s
	Train Loss: 2.303 | Train Acc: 9.91%
	 Val. Loss: 2.304 |  Val. Acc: 10.25%


Epoch: 03 | Epoch Time: 0m 23s
	Train Loss: 2.303 | Train Acc: 9.95%
	 Val. Loss: 2.303 |  Val. Acc: 9.63%


Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 2.304 | Train Acc: 9.78%
	 Val. Loss: 2.303 |  Val. Acc: 10.19%


...

Epoch: 28 | Epoch Time: 0m 18s
	Train Loss: 2.303 | Train Acc: 10.05%
	 Val. Loss: 2.303 |  Val. Acc: 10.05%


Epoch: 29 | Epoch Time: 0m 18s
	Train Loss: 2.303 | Train Acc: 9.90%
	 Val. Loss: 2.303 |  Val. Acc: 10.15%


Epoch: 30 | Epoch Time: 0m 18s
	Train Loss: 2.303 | Train Acc: 9.87%
	 Val. Loss: 2.303 |  Val. Acc: 10.25%


Epoch: 31 | Epoch Time: 0m 18s
	Train Loss: 2.303 | Train Acc: 10.09%
	 Val. Loss: 2.303 |  Val. Acc: 9.63%


Epoch: 32 | Epoch Time: 0m 18s
	Train Loss: 2.303 | Train Acc: 10.17%
	 Val. Loss: 2.302 |  Val. Acc: 10.56%


In [22]:
model.load_state_dict(torch.load('../results/LeNet-02.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 2.303 | Test Acc: 9.95%


## Comparing Optimizers

### Model_03

 - Batch Size : 64
 - Loss function: Cross-Entropy Loss
 - Optimizer: Adam
 - Activation function: ReLu
 - Epoch : 32

In [23]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [24]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [26]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-03.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-03-train-stats")

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 1.649 | Train Acc: 39.65%
	 Val. Loss: 1.470 |  Val. Acc: 46.60%


Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 1.365 | Train Acc: 50.65%
	 Val. Loss: 1.327 |  Val. Acc: 51.34%


Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 1.251 | Train Acc: 55.26%
	 Val. Loss: 1.235 |  Val. Acc: 55.28%


Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 1.174 | Train Acc: 58.13%
	 Val. Loss: 1.210 |  Val. Acc: 56.03%


...

Epoch: 28 | Epoch Time: 0m 18s
	Train Loss: 0.506 | Train Acc: 81.82%
	 Val. Loss: 1.403 |  Val. Acc: 61.14%


Epoch: 29 | Epoch Time: 0m 18s
	Train Loss: 0.495 | Train Acc: 82.28%
	 Val. Loss: 1.412 |  Val. Acc: 61.35%


Epoch: 30 | Epoch Time: 0m 18s
	Train Loss: 0.477 | Train Acc: 83.09%
	 Val. Loss: 1.521 |  Val. Acc: 59.91%


Epoch: 31 | Epoch Time: 0m 18s
	Train Loss: 0.471 | Train Acc: 83.09%
	 Val. Loss: 1.466 |  Val. Acc: 60.50%


Epoch: 32 | Epoch Time: 0m 18s
	Train Loss: 0.451 | Train Acc: 83.69%
	 Val. Loss: 1.539 |  Val. Acc: 60.54%


In [27]:
model.load_state_dict(torch.load('../results/LeNet-03.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.074 | Test Acc: 63.18%


## Comparing Batch Sizes

### Model_04

 - Batch Size : 4
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: ReLu
 - Epoch : 32

In [28]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [29]:
BATCH_SIZE = 4
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [31]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-04.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-04-train-stats")

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 53s
	Train Loss: 1.702 | Train Acc: 36.93%
	 Val. Loss: 1.561 |  Val. Acc: 43.14%


Epoch: 02 | Epoch Time: 0m 54s
	Train Loss: 1.365 | Train Acc: 51.10%
	 Val. Loss: 1.351 |  Val. Acc: 50.72%


Epoch: 03 | Epoch Time: 0m 52s
	Train Loss: 1.230 | Train Acc: 56.29%
	 Val. Loss: 1.286 |  Val. Acc: 54.40%


Epoch: 04 | Epoch Time: 0m 53s
	Train Loss: 1.136 | Train Acc: 59.87%
	 Val. Loss: 1.194 |  Val. Acc: 57.64%


...

Epoch: 28 | Epoch Time: 1m 5s
	Train Loss: 0.621 | Train Acc: 78.23%
	 Val. Loss: 1.640 |  Val. Acc: 56.76%


Epoch: 29 | Epoch Time: 1m 4s
	Train Loss: 0.614 | Train Acc: 78.60%
	 Val. Loss: 1.598 |  Val. Acc: 58.96%


Epoch: 30 | Epoch Time: 1m 6s
	Train Loss: 0.606 | Train Acc: 78.72%
	 Val. Loss: 1.744 |  Val. Acc: 57.30%


Epoch: 31 | Epoch Time: 1m 4s
	Train Loss: 0.614 | Train Acc: 78.74%
	 Val. Loss: 1.581 |  Val. Acc: 58.54%


Epoch: 32 | Epoch Time: 1m 6s
	Train Loss: 0.607 | Train Acc: 79.16%
	 Val. Loss: 1.702 |  Val. Acc: 56.56%


In [32]:
model.load_state_dict(torch.load('../results/LeNet-04.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.130 | Test Acc: 60.99%


### Model_05

 - Batch Size : 16
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: ReLu
 - Epoch : 32

In [33]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [34]:
BATCH_SIZE = 16
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [36]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-05.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-05-train-stats")

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 28s
	Train Loss: 2.209 | Train Acc: 17.26%
	 Val. Loss: 1.906 |  Val. Acc: 31.65%


Epoch: 02 | Epoch Time: 0m 28s
	Train Loss: 1.683 | Train Acc: 38.28%
	 Val. Loss: 1.555 |  Val. Acc: 43.43%


Epoch: 03 | Epoch Time: 0m 27s
	Train Loss: 1.470 | Train Acc: 46.33%
	 Val. Loss: 1.446 |  Val. Acc: 47.02%


Epoch: 04 | Epoch Time: 0m 27s
	Train Loss: 1.358 | Train Acc: 51.04%
	 Val. Loss: 1.344 |  Val. Acc: 51.60%


...

Epoch: 28 | Epoch Time: 0m 26s
	Train Loss: 0.518 | Train Acc: 81.53%
	 Val. Loss: 1.362 |  Val. Acc: 60.74%


Epoch: 29 | Epoch Time: 0m 26s
	Train Loss: 0.500 | Train Acc: 82.20%
	 Val. Loss: 1.397 |  Val. Acc: 62.10%


Epoch: 30 | Epoch Time: 0m 26s
	Train Loss: 0.481 | Train Acc: 82.81%
	 Val. Loss: 1.436 |  Val. Acc: 61.10%


Epoch: 31 | Epoch Time: 0m 26s
	Train Loss: 0.465 | Train Acc: 83.28%
	 Val. Loss: 1.500 |  Val. Acc: 61.10%


Epoch: 32 | Epoch Time: 0m 26s
	Train Loss: 0.451 | Train Acc: 84.07%
	 Val. Loss: 1.539 |  Val. Acc: 61.02%


In [37]:
model.load_state_dict(torch.load('../results/LeNet-05.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.057 | Test Acc: 62.78%


### Model_06

 - Batch Size : 32
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: ReLu
 - Epoch : 32

In [38]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [39]:
BATCH_SIZE = 32
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [41]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-06.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-06-train-stats")

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 22s
	Train Loss: 2.252 | Train Acc: 17.42%
	 Val. Loss: 2.067 |  Val. Acc: 24.58%


Epoch: 02 | Epoch Time: 0m 21s
	Train Loss: 1.931 | Train Acc: 30.04%
	 Val. Loss: 1.809 |  Val. Acc: 34.89%


Epoch: 03 | Epoch Time: 0m 20s
	Train Loss: 1.669 | Train Acc: 39.56%
	 Val. Loss: 1.575 |  Val. Acc: 41.72%


Epoch: 04 | Epoch Time: 0m 20s
	Train Loss: 1.516 | Train Acc: 45.02%
	 Val. Loss: 1.474 |  Val. Acc: 46.32%


...

Epoch: 28 | Epoch Time: 0m 20s
	Train Loss: 0.684 | Train Acc: 75.72%
	 Val. Loss: 1.096 |  Val. Acc: 63.87%


Epoch: 29 | Epoch Time: 0m 20s
	Train Loss: 0.667 | Train Acc: 76.47%
	 Val. Loss: 1.115 |  Val. Acc: 63.54%


Epoch: 30 | Epoch Time: 0m 20s
	Train Loss: 0.650 | Train Acc: 77.27%
	 Val. Loss: 1.131 |  Val. Acc: 63.67%


Epoch: 31 | Epoch Time: 0m 20s
	Train Loss: 0.628 | Train Acc: 77.95%
	 Val. Loss: 1.141 |  Val. Acc: 63.99%


Epoch: 32 | Epoch Time: 0m 20s
	Train Loss: 0.613 | Train Acc: 78.23%
	 Val. Loss: 1.174 |  Val. Acc: 63.34%


In [42]:
model.load_state_dict(torch.load('../results/LeNet-06.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.036 | Test Acc: 65.16%


## Comparing Learning Rates

### Model_07

 - Batch Size : 64
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: ReLu
 - Epoch : 32
 - Learning Rate : 0.1

In [43]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [44]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum = 0.9)

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [46]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-07.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-07-train-stats")


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 1.861 | Train Acc: 31.10%
	 Val. Loss: 1.989 |  Val. Acc: 27.37%


Epoch: 02 | Epoch Time: 0m 19s
	Train Loss: 1.780 | Train Acc: 34.66%
	 Val. Loss: 1.733 |  Val. Acc: 38.21%


Epoch: 03 | Epoch Time: 0m 19s
	Train Loss: 1.794 | Train Acc: 34.61%
	 Val. Loss: 1.766 |  Val. Acc: 37.32%


Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 1.780 | Train Acc: 35.89%
	 Val. Loss: 1.813 |  Val. Acc: 35.52%


...

Epoch: 28 | Epoch Time: 0m 18s
	Train Loss: 2.017 | Train Acc: 21.23%
	 Val. Loss: 2.007 |  Val. Acc: 20.61%


Epoch: 29 | Epoch Time: 0m 19s
	Train Loss: 2.041 | Train Acc: 20.93%
	 Val. Loss: 2.050 |  Val. Acc: 20.73%


Epoch: 30 | Epoch Time: 0m 19s
	Train Loss: 2.066 | Train Acc: 20.35%
	 Val. Loss: 2.048 |  Val. Acc: 20.09%


Epoch: 31 | Epoch Time: 0m 18s
	Train Loss: 2.153 | Train Acc: 16.33%
	 Val. Loss: 2.228 |  Val. Acc: 13.57%


Epoch: 32 | Epoch Time: 0m 19s
	Train Loss: 2.226 | Train Acc: 13.31%
	 Val. Loss: 2.134 |  Val. Acc: 15.98%


In [47]:
model.load_state_dict(torch.load('../results/LeNet-07.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.735 | Test Acc: 38.65%


### Model_08

 - Batch Size : 64
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: ReLu
 - Epoch : 32
 - Learning Rate : 0.01

In [48]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [49]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [51]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-08.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-08-train-stats")


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 1.871 | Train Acc: 30.56%
	 Val. Loss: 1.537 |  Val. Acc: 43.97%


Epoch: 02 | Epoch Time: 0m 17s
	Train Loss: 1.449 | Train Acc: 47.17%
	 Val. Loss: 1.508 |  Val. Acc: 46.22%


Epoch: 03 | Epoch Time: 0m 17s
	Train Loss: 1.284 | Train Acc: 53.70%
	 Val. Loss: 1.251 |  Val. Acc: 54.83%


Epoch: 04 | Epoch Time: 0m 17s
	Train Loss: 1.182 | Train Acc: 57.82%
	 Val. Loss: 1.222 |  Val. Acc: 56.61%


...

Epoch: 28 | Epoch Time: 0m 25s
	Train Loss: 0.496 | Train Acc: 82.29%
	 Val. Loss: 1.626 |  Val. Acc: 58.88%


Epoch: 29 | Epoch Time: 0m 26s
	Train Loss: 0.471 | Train Acc: 83.11%
	 Val. Loss: 1.715 |  Val. Acc: 58.31%


Epoch: 30 | Epoch Time: 0m 25s
	Train Loss: 0.469 | Train Acc: 83.37%
	 Val. Loss: 1.881 |  Val. Acc: 56.98%


Epoch: 31 | Epoch Time: 0m 24s
	Train Loss: 0.457 | Train Acc: 83.58%
	 Val. Loss: 1.866 |  Val. Acc: 58.45%


Epoch: 32 | Epoch Time: 0m 24s
	Train Loss: 0.446 | Train Acc: 84.09%
	 Val. Loss: 1.879 |  Val. Acc: 57.79%


In [52]:
model.load_state_dict(torch.load('../results/LeNet-08.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.111 | Test Acc: 62.54%


### Model_09

 - Batch Size : 64
 - Loss function: Cross-Entropy Loss
 - Optimizer: Stochastic Gradient Descent (SGD)
 - Activation function: ReLu
 - Epoch : 32
 - Learning Rate : 0.05

In [53]:
model = LeNet()
print(f'The model has {count_parameters(model):,} trainable parameters')
model

The model has 62,006 trainable parameters


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [54]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = create_data_loaders(train_data,
                                                                    valid_data,
                                                                    test_data,
                                                                    BATCH_SIZE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9)

In [55]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is : {device}")

Current device is : cuda


In [56]:
EPOCHS = 32

train_losses = []
valid_losses = []

valid_accs = []
train_accs = []

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)

for epoch in trange(EPOCHS):

  start_time = time.monotonic()

  train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  train_accs.append(train_acc)
  valid_accs.append(valid_acc)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), '../results/LeNet-09.pt')

  end_time = time.monotonic()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

train_stats = df_results(EPOCHS, train_losses, valid_losses, train_accs, valid_accs, "LeNet-09-train-stats")


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: 1.717 | Train Acc: 36.80%
	 Val. Loss: 1.485 |  Val. Acc: 44.50%


Epoch: 02 | Epoch Time: 0m 25s
	Train Loss: 1.437 | Train Acc: 48.33%
	 Val. Loss: 1.499 |  Val. Acc: 46.30%


Epoch: 03 | Epoch Time: 0m 23s
	Train Loss: 1.360 | Train Acc: 52.07%
	 Val. Loss: 1.436 |  Val. Acc: 50.49%


Epoch: 04 | Epoch Time: 0m 24s
	Train Loss: 1.331 | Train Acc: 53.69%
	 Val. Loss: 1.386 |  Val. Acc: 51.13%


...

Epoch: 28 | Epoch Time: 0m 23s
	Train Loss: 1.444 | Train Acc: 53.80%
	 Val. Loss: 1.628 |  Val. Acc: 47.86%


Epoch: 29 | Epoch Time: 0m 25s
	Train Loss: 1.459 | Train Acc: 53.21%
	 Val. Loss: 1.765 |  Val. Acc: 46.02%


Epoch: 30 | Epoch Time: 0m 23s
	Train Loss: 1.492 | Train Acc: 51.86%
	 Val. Loss: 1.681 |  Val. Acc: 48.56%


Epoch: 31 | Epoch Time: 0m 25s
	Train Loss: 1.522 | Train Acc: 51.36%
	 Val. Loss: 1.758 |  Val. Acc: 42.48%


Epoch: 32 | Epoch Time: 0m 24s
	Train Loss: 1.594 | Train Acc: 47.94%
	 Val. Loss: 2.022 |  Val. Acc: 38.05%


In [57]:
model.load_state_dict(torch.load('../results/LeNet-09.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.360 | Test Acc: 53.98%
